In [1]:
import requests, zipfile, json, os, sqlite3
import ipywidgets as widgets
from IPython.display import display
import pickle
%run print_map.ipynb


In [2]:
#Download and unzip the manifest

manifest_url = 'https://www.bungie.net/Platform/Destiny2/Manifest/'

#get the manifest location from the json
r = requests.get(manifest_url)
manifest = r.json()
mani_url = 'http://www.bungie.net'+manifest['Response']['mobileWorldContentPaths']['en']

#Download the file, write it to 'MANZIP'
r = requests.get(mani_url)
with open("MANZIP", "wb") as zip:
    zip.write(r.content)
print("Download Complete!")

#Extract the file contents, and rename the extracted file
# to 'Manifest.content'
with zipfile.ZipFile('MANZIP') as zip:
    name = zip.namelist()
    zip.extractall()
os.rename(name[0], 'Manifest.content')
print('Unzipped!')

Download Complete!
Unzipped!


In [3]:
#list of all the tables used in this code
tables = [
    'DestinyPresentationNodeDefinition',
    'DestinyRecordDefinition',
    'DestinyInventoryItemDefinition',
    'DestinyObjectiveDefinition',
    'DestinyItemCategoryDefinition',
    'DestinyVendorDefinition',
    'DestinyInventoryBucketDefinition'
]

#initializes the manifest dictionary
manifest = {}

#queries the manifest SQL data for each table
for table in tables:
    con = sqlite3.connect('manifest.content')
    cur = con.cursor()
    cur.execute('SELECT json from '+ table)
    items = cur.fetchall()
    manifest[table] = [json.loads(item[0]) for item in items]
    
with open('manifest.data', 'wb') as filehandle:
    pickle.dump(manifest, filehandle)    

In [4]:
#initialize nodes dictionary for condensed use
nodes = {}

#loops through each node and condenses it to the nodes dictionary
for item in manifest['DestinyPresentationNodeDefinition']:
    nodes[item['hash']] = {}
    nodes[item['hash']]['name'] = item['displayProperties']['name']
    if len(item['parentNodeHashes'])>0:
        nodes[item['hash']]['parent'] = item['parentNodeHashes'][0]
    else:
        nodes[item['hash']]['parent'] = 'none'
        
#returns the node path
def get_node_path(node):
    parent_path = ''
    if nodes[node]['parent'] != 'none':
        parent_path = get_node_path(nodes[node]['parent']) + "/"
    path = parent_path + nodes[node]['name']
    return path

objectives = {}
for objective in manifest['DestinyObjectiveDefinition']:
    objectives[objective['hash']] = {
        'name': objective['displayProperties']['name'],
        'description': objective['progressDescription'],
        'completionValue' : objective['completionValue'],
        'progress': -1,
        'complete': False
    }
    


In [5]:
#initialize lists
triumphs = {}
triumphs_ignored = []
expire_phrases = [   #used to mark if a triumph has expired
    'Available only during Moments of Triumph 2019.',
    'Available only during Season of the Forge.',
    'Available only during this year\'s Dawning.',
    'Available only during the 2019 Revelry.',
    'Available only during Season 7',
    'Available only during Season 6, the Season of the Drifter.',
    'Available only during Crimson Days',
    #'Available only during Season of the Undying.'
    #'Available only during Season 8, Season of the Undying.'
    ]         

#loop through each triumph
for record in manifest['DestinyRecordDefinition']:
    
    #checks to see if the record should be ignored
    ignore = False
    if record['displayProperties']['name'] == '':
        ignore = True
    if record['displayProperties']['description'] == '' and 'loreHash' not in record:
        ignore = True
    if 'completionInfo' not in record:
        ignore = True
    if 'presentationInfo' in record:
        if len(record['presentationInfo']['parentPresentationNodeHashes']) < 1:
            ignore = True
    else:
        ignore = True
    
    #adds the triump to the triumph lists
    if ignore:
        triumphs_ignored.append(record)
    else:
        
        #makes the triumph dictionary more readable
        triumph = {}
        
        #sets the name
        triumph['name'] = record['displayProperties']['name']
        
        #finds the path (triumphs first, seals second if they are listed in two areas)
        triumph['path'] = ''
        for node in record['presentationInfo']['parentPresentationNodeHashes']:
            path = get_node_path(node) + '/' + triumph['name']
            if 'Triumphs' in path:
                triumph['path'] = path
            elif 'Seals' in path and 'Triumphs' not in triumph['path']:
                triumph['path'] = path     
        
        #adds a description
        triumph['description'] = record['displayProperties']['description']
        
        #TODO: Add objective to the triumphs
        
        #adds a triumph score value
        #triumph['value'] = record['completionInfo']['ScoreValue']
        
        #checks to see if the triumph has expired
        if record['expirationInfo']['hasExpiration']:
            if record['expirationInfo']['description'] in expire_phrases:
                triumph['expired'] = True
            else:
                triumph['expired'] = False
        else:
            triumph['expired'] = False
        
        #adds ojective details
        triumph['objectives'] = {}
        for objective in record['objectiveHashes']:
            triumph['objectives'][objective] = objectives[objective]
        for objective in record['intervalInfo']['intervalObjectives']:
            hash = objective['intervalObjectiveHash']
            triumph['objectives'][hash] = objectives[hash]

        #appends the condensed triumph information to the triumphs list
        triumphs[record['hash']] = triumph
            
with open('triumphs.data', 'wb') as filehandle:
    pickle.dump(triumphs, filehandle) 

In [6]:
pursuits = {}

for item in manifest['DestinyInventoryItemDefinition']:
    if item['itemType'] in [26, 15, 12]:
        pursuit = {}
        pursuit['name'] = item['displayProperties']['name']
        pursuit['description'] = item['displayProperties']['description']
        #adds ojective details
        pursuit['objectives'] = {}
        if 'objectives' in item:
            for objectiveHash in item['objectives']['objectiveHashes']:
                pursuit['objectives'][objectiveHash] = objectives[objectiveHash]
        if item['itemType'] == 26:
            pursuit['type'] = 'Bounty'
        else:
            pursuit['type'] = 'Quest'
        pursuits[item['hash']] = pursuit
        
important_vendors = ['919809084', '3982706173', '396892126', '3347378076', 
                     '1062861569', '1265988377', '1576276905', '69482069', 
                     '1976548992', '3603221665', '2255782930', '672118013', 
                     '895295461', '2190858386', '460529231', '3190557734', 
                     '2398407866', '1735426333', '997622907', '2067572909', 
                     '1841717884', '863940356', '248695599', '1796504621', 
                     '2917531897', '880202832', '1616085565', '3411552308', 
                     '2894222926', '3361454721']
        
vendors = {}
for vendor in manifest['DestinyVendorDefinition']:
    vendors[vendor['hash']] = {}
    vendors[vendor['hash']]['name'] = vendor['displayProperties']['name']
    vendors[vendor['hash']]['pursuits'] = []
    if str(vendor['hash']) in important_vendors:
        for item in vendor['itemList']:
            if item['itemHash'] in pursuits:
                pursuits[item['itemHash']]['vendor'] = vendor['displayProperties']['name']
                vendors[vendor['hash']]['pursuits'].append(item['itemHash'])
    
    
with open('vendors.data', 'wb') as filehandle:
    pickle.dump(vendors, filehandle)
with open('pursuits.data', 'wb') as filehandle:
    pickle.dump(pursuits, filehandle) 
